<a href="https://colab.research.google.com/github/FKz11/NLP/blob/main/lesson_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson-8

### Библиотеки:

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

### Данные:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Отзывы за лето.csv', encoding="windows-1251", sep=';')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Rating', axis=1), df['Rating'], random_state=42)
X_train.head()

,Content,Date
6926,Отличное приложение)))),2017-08-04
6735,Олекум,2017-08-04
149,"Хорошее приложение, но «антивирус» очень загру...",2017-08-14
3069,Потянет,2017-08-09
17455,Ужасное приложение положил деньги и они испари...,2017-07-11


In [5]:
y_train.value_counts()

5    10933
1     1699
4     1627
3      677
2      558
Name: Rating, dtype: int64

In [6]:
disbalance=(y_train.value_counts().sort_index().max()/y_train.value_counts().sort_index()).values
disbalance

array([ 6.43496174, 19.59318996, 16.14918759,  6.71972956,  1.        ])

### Предобработка

Предобработку возьмём с прошлого дз, так как данные те же

In [7]:
!pip install -qq stop_words
!pip install -qq pymorphy2

     |████████████████████████████████| 55 kB 4.4 MB/s 
     |████████████████████████████████| 8.2 MB 54.1 MB/s 


In [8]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = [morpher.parse(word)[0].normal_form for word in nltk.word_tokenize(txt) if word not in sw]
    return " ".join(txt)

X_train['Content_preprocess'] = X_train['Content'].apply(preprocess_text)
X_test['Content_preprocess'] = X_test['Content'].apply(preprocess_text)
X_train.head()

,Content,Date,Content_preprocess
6926,Отличное приложение)))),2017-08-04,отличный приложение
6735,Олекум,2017-08-04,олекума
149,"Хорошее приложение, но «антивирус» очень загру...",2017-08-14,хороший приложение « антивирус » загружать сис...
3069,Потянет,2017-08-09,потянуть
17455,Ужасное приложение положил деньги и они испари...,2017-07-11,ужасный приложение положить деньга испариться ...


In [11]:
from collections import Counter

In [12]:
corpus = (" ".join(X_train['Content_preprocess'])).split()
corpus_count = Counter(corpus)
corpus_count.most_common(10)

[('приложение', 4849),
 ('удобно', 2495),
 ('работать', 1483),
 ('удобный', 1336),
 ('отлично', 968),
 ('нравиться', 858),
 ('хороший', 770),
 ('отличный', 758),
 ('телефон', 750),
 ('супер', 628)]

In [13]:
max_words_true = len(corpus_count)
max_words_true

9064

In [14]:
MAX_WORDS = max_words_true
MAX_WORDS

9064

In [15]:
vocabulary = {j:i for i, (j, _) in enumerate(corpus_count.most_common()[:MAX_WORDS], 1)}
list(vocabulary.items())[:10]

[('приложение', 1),
 ('удобно', 2),
 ('работать', 3),
 ('удобный', 4),
 ('отлично', 5),
 ('нравиться', 6),
 ('хороший', 7),
 ('отличный', 8),
 ('телефон', 9),
 ('супер', 10)]

In [16]:
max_len_true = max([len(txt) for txt in X_train['Content_preprocess']])
max_len_true

877

In [17]:
np.mean([len(txt) for txt in X_train['Content_preprocess']])

40.31754227442881

In [18]:
MAX_LEN = 50
MAX_LEN

50

In [19]:
X_train['Content_sequence'] = X_train['Content_preprocess'].apply(lambda text: [vocabulary.get(word, 0) for word in text.split()][:MAX_LEN]+[0]*(MAX_LEN-len(text.split())))
X_test['Content_sequence'] = X_test['Content_preprocess'].apply(lambda text: [vocabulary.get(word, 0) for word in text.split()][:MAX_LEN]+[0]*(MAX_LEN-len(text.split())))
X_train.head()

,Content,Date,Content_preprocess,Content_sequence
6926,Отличное приложение)))),2017-08-04,отличный приложение,"[8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6735,Олекум,2017-08-04,олекума,"[3391, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
149,"Хорошее приложение, но «антивирус» очень загру...",2017-08-14,хороший приложение « антивирус » загружать сис...,"[7, 1, 1016, 16, 1017, 595, 199, 0, 0, 0, 0, 0..."
3069,Потянет,2017-08-09,потянуть,"[2426, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17455,Ужасное приложение положил деньги и они испари...,2017-07-11,ужасный приложение положить деньга испариться ...,"[211, 1, 534, 36, 3392, 773, 356, 472, 1633, 8..."


In [20]:
X_train_array = np.array([[j for j in i] for i in X_train['Content_sequence']])
X_test_array = np.array([[j for j in i] for i in X_test['Content_sequence']])
X_train_array[0][:10]

array([8, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [21]:
y_train_array = y_train-1
y_test_array = y_test-1
y_train_array.head()

6926     4
6735     4
149      3
3069     3
17455    3
Name: Rating, dtype: int64

In [22]:
dict_metrics = {}

## 1. построить свёрточные архитектуры

#### CNN_ker3

In [23]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout, BatchNormalization, Conv1D, LSTM, GRU, SimpleRNN, GlobalAveragePooling1D

In [24]:
from sklearn.metrics import classification_report

In [25]:
def go_model(model):
  print(model.summary())
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              loss_weights=disbalance,
              optimizer='adam',
              metrics=['accuracy'])
  with tf.device("GPU:0"):
    hh = model.fit(X_train_array, y_train_array, validation_data=(X_test_array, y_test_array), epochs=10)
  print(classification_report(y_test_array, np.argmax(model.predict(X_test_array), axis=1)))
  return hh.history['val_accuracy'][-1]

In [ ]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    Conv1D(embedding_dim*2, 3),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
dict_metrics['CNN_ker3'] = go_model(model)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 300)         2719500   
                                                                 
 conv1d_9 (Conv1D)           (None, None, 600)         540600    
                                                                 
 global_max_pooling1d_6 (Glo  (None, 600)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_18 (Dense)            (None, 512)               307712    
                                                                 
 dense_19 (Dense)            (None, 256)               131328    
                                                                 
 dense_20 (Dense)            (None, 5)                 1285      
                                                      

#### CNN_ker2_ker3

In [ ]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    Conv1D(embedding_dim*2, 2),
    Conv1D(embedding_dim*4, 3),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
dict_metrics['CNN_ker2_ker3'] = go_model(model)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 300)         2719500   
                                                                 
 conv1d_10 (Conv1D)          (None, None, 600)         360600    
                                                                 
 conv1d_11 (Conv1D)          (None, None, 1200)        2161200   
                                                                 
 global_max_pooling1d_7 (Glo  (None, 1200)             0         
 balMaxPooling1D)                                                
                                                                 
 dense_21 (Dense)            (None, 512)               614912    
                                                                 
 dense_22 (Dense)            (None, 256)               131328    
                                                      

Увелечение количества свёрток не улучшило результат

## 2. построить различные архитектуры с RNN

#### RNN

In [ ]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim),
    SimpleRNN(embedding_dim),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
dict_metrics['RNN_rnn'] = go_model(model)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, None, 300)         2719500   
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 300)               180300    
                                                                 
 dense_30 (Dense)            (None, 512)               154112    
                                                                 
 dense_31 (Dense)            (None, 256)               131328    
                                                                 
 dense_32 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 3,186,525
Trainable params: 3,186,525
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
485/485 [==================

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


У RNN не получается нормально обучаться, так как длина последовательности = 50 и она может нормально анализировать олько какие то последние n токенов, но они то как раз в большинстве случаев равны 0 и не несут смысла.

Но это можно легко исправить добавив в слой Embedding mask_zero=True. Теперь эти последние нулевые токены не будут учитываться RNN, и она будет обучаться так же не глубоко, по последним токенам, но хотя бы не по нулевым токенам, а по тем, которые несут смысл.

In [ ]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    SimpleRNN(embedding_dim),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
dict_metrics['RNN_rnn'] = go_model(model)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, None, 300)         2719500   
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 300)               180300    
                                                                 
 dense_33 (Dense)            (None, 512)               154112    
                                                                 
 dense_34 (Dense)            (None, 256)               131328    
                                                                 
 dense_35 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 3,186,525
Trainable params: 3,186,525
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
485/485 [==================

RNN всё равно предаёт большее значение последним словам, так как мы используем только последний выход. Попробуем это изменить, анализируя каждые выходы.

#### RNN_rnn_seq

In [ ]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    SimpleRNN(embedding_dim, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
dict_metrics['RNN_rnn_seq'] = go_model(model)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, None, 300)         2719500   
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, None, 300)         180300    
                                                                 
 global_max_pooling1d_8 (Glo  (None, 300)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_39 (Dense)            (None, 512)               154112    
                                                                 
 dense_40 (Dense)            (None, 256)               131328    
                                                                 
 dense_41 (Dense)            (None, 5)                 1285      
                                                     

Как видно, с усреднением результаты оказались немного лучше

#### RNN_lstm

In [ ]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    LSTM(embedding_dim),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
dict_metrics['RNN_lstm'] = go_model(model)

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, None, 300)         2719500   
                                                                 
 lstm (LSTM)                 (None, 300)               721200    
                                                                 
 dense_42 (Dense)            (None, 512)               154112    
                                                                 
 dense_43 (Dense)            (None, 256)               131328    
                                                                 
 dense_44 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 3,727,425
Trainable params: 3,727,425
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
485/485 [==================

Результаты получились хуже из-за сильного переобучения, так как слой lstm гораздо тяжелее rnn

#### RNN_lstm_seq

In [ ]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    LSTM(embedding_dim, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
dict_metrics['RNN_lstm_seq'] = go_model(model)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, None, 300)         2719500   
                                                                 
 lstm_4 (LSTM)               (None, None, 300)         721200    
                                                                 
 global_max_pooling1d_11 (Gl  (None, 300)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_54 (Dense)            (None, 512)               154112    
                                                                 
 dense_55 (Dense)            (None, 256)               131328    
                                                                 
 dense_56 (Dense)            (None, 5)                 1285      
                                                     

Пока лучший результат

#### RNN_gru

In [ ]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    GRU(embedding_dim),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
dict_metrics['RNN_gru'] = go_model(model)

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, None, 300)         2719500   
                                                                 
 gru (GRU)                   (None, 300)               541800    
                                                                 
 dense_57 (Dense)            (None, 512)               154112    
                                                                 
 dense_58 (Dense)            (None, 256)               131328    
                                                                 
 dense_59 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 3,548,025
Trainable params: 3,548,025
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
485/485 [==================

gru оказалась ни быстрее, ни лучше

#### RNN_lstm_seq_2lay

In [ ]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    LSTM(embedding_dim, return_sequences=True),
    LSTM(embedding_dim, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
dict_metrics['RNN_lstm_seq_2lay'] = go_model(model)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, None, 300)         2719500   
                                                                 
 lstm_5 (LSTM)               (None, None, 300)         721200    
                                                                 
 lstm_6 (LSTM)               (None, None, 300)         721200    
                                                                 
 global_max_pooling1d_12 (Gl  (None, 300)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_60 (Dense)            (None, 512)               154112    
                                                                 
 dense_61 (Dense)            (None, 256)               131328    
                                                     

Метрики не улучшились, а вот обучается ровно в 2 раза дольше

Пустим второй слой в обратном направлении, чтобы анализировать текст еще и в обратном направлении, это может помочь достать какие то новые признаки

#### RNN_lstm_there_back

In [28]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    LSTM(embedding_dim, return_sequences=True),
    LSTM(embedding_dim, return_sequences=True, go_backwards=True),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [29]:
dict_metrics['RNN_lstm_there_back'] = go_model(model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         2719500   
                                                                 
 lstm (LSTM)                 (None, None, 300)         721200    
                                                                 
 lstm_1 (LSTM)               (None, None, 300)         721200    
                                                                 
 global_max_pooling1d (Globa  (None, 300)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 512)               154112    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                        

Результаты похожие

Видно, что основная проблема, это переобучение, попробуем это исправить с помощью drop

#### RNN_lstm_seq_drop

In [33]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    LSTM(embedding_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [34]:
dict_metrics['RNN_lstm_seq_drop'] = go_model(model)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 300)         2719500   
                                                                 
 lstm_3 (LSTM)               (None, None, 300)         721200    
                                                                 
 global_max_pooling1d_3 (Glo  (None, 300)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_9 (Dense)             (None, 512)               154112    
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                                 
 dense_11 (Dense)            (None, 5)                 1285      
                                                      

Переобучение сохранилось

## 3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

#### CNN_RNN

In [35]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    Conv1D(embedding_dim*2, 3),
    LSTM(embedding_dim*2, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [36]:
dict_metrics['CNN_RNN'] = go_model(model)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 300)         2719500   
                                                                 
 conv1d (Conv1D)             (None, None, 600)         540600    
                                                                 
 lstm_4 (LSTM)               (None, None, 600)         2882400   
                                                                 
 global_max_pooling1d_4 (Glo  (None, 600)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_12 (Dense)            (None, 512)               307712    
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                      

#### RNN_CNN

In [37]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    LSTM(embedding_dim*2, return_sequences=True),
    Conv1D(embedding_dim*2, 3),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [38]:
dict_metrics['RNN_CNN'] = go_model(model)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 300)         2719500   
                                                                 
 lstm_5 (LSTM)               (None, None, 600)         2162400   
                                                                 
 conv1d_1 (Conv1D)           (None, None, 600)         1080600   
                                                                 
 global_max_pooling1d_5 (Glo  (None, 600)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_15 (Dense)            (None, 512)               307712    
                                                                 
 dense_16 (Dense)            (None, 256)               131328    
                                                      

#### baseline

In [39]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [40]:
dict_metrics['baseline'] = go_model(model)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 300)         2719500   
                                                                 
 global_max_pooling1d_6 (Glo  (None, 300)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_18 (Dense)            (None, 512)               154112    
                                                                 
 dense_19 (Dense)            (None, 256)               131328    
                                                                 
 dense_20 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 3,006,225
Trainable params: 3,006,225
Non-trainable params: 0
____________________________________________

Как видно, CNN и RNN едва ли как то влияют

#### LogisticRegression

In [218]:
from sklearn.linear_model import LogisticRegression

In [219]:
lr = LogisticRegression()

In [220]:
lr.fit(X_train_array, y_train_array)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [221]:
print(classification_report(y_test_array, lr.predict(X_test_array)))

              precision    recall  f1-score   support

           0       0.19      0.34      0.25       320
           1       0.01      0.17      0.02        12
           2       0.03      0.11      0.04        57
           3       0.00      0.04      0.00        28
           4       0.98      0.75      0.85      4748

    accuracy                           0.71      5165
   macro avg       0.24      0.28      0.23      5165
weighted avg       0.91      0.71      0.80      5165



In [223]:
dict_metrics['log_reg'] = np.mean(lr.predict(X_test_array)==y_test_array)

Самая простая логичтическая регресия так же справляется, но она почти не предсказывает промежуточные классы 1,2,3, поэтому метрика получилась хуже.

### Другой взгляд на задачу:

Наши метки 0,1,2,3,4 явно друг от друга зависят, 0<1<2<3<4, поэтому лучше решать эту задачу как задачу регресии и потом округлять ответ до целого числа, так мы будем наказывать модель больше, если она вместо 4 предсказала 0, и меньше, если 3, что соответсвует действительности.

In [225]:
import keras.backend as K

In [242]:
def reg_acc(y_true, y_pred):
    return K.mean(y_true==K.round(K.clip(y_pred, 0, 4)))

In [243]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    Conv1D(embedding_dim*2, 3),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=1, activation='relu')
])

In [244]:
print(model.summary())
model.compile(loss=tf.keras.losses.MeanAbsoluteError(),
              loss_weights=disbalance,
              optimizer='adam',
              metrics=[reg_acc])

Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_47 (Embedding)    (None, None, 300)         2719500   
                                                                 
 conv1d_36 (Conv1D)          (None, None, 600)         540600    
                                                                 
 global_max_pooling1d_47 (Gl  (None, 600)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_141 (Dense)           (None, 512)               307712    
                                                                 
 dense_142 (Dense)           (None, 256)               131328    
                                                                 
 dense_143 (Dense)           (None, 1)                 257       
                                                     

In [245]:
with tf.device("GPU:0"):
    hh = model.fit(X_train_array, y_train_array, validation_data=(X_test_array, y_test_array), epochs=10)

Epoch 1/10
485/485 [==============================] - 5s 8ms/step - loss: 6.0018 - reg_acc: 0.6967 - val_loss: 6.1943 - val_reg_acc: 0.7508
Epoch 2/10
485/485 [==============================] - 3s 7ms/step - loss: 4.3147 - reg_acc: 0.7674 - val_loss: 4.5496 - val_reg_acc: 0.7531
Epoch 3/10
485/485 [==============================] - 3s 7ms/step - loss: 3.6960 - reg_acc: 0.7921 - val_loss: 4.9848 - val_reg_acc: 0.7470
Epoch 4/10
485/485 [==============================] - 4s 8ms/step - loss: 3.1311 - reg_acc: 0.8141 - val_loss: 4.8063 - val_reg_acc: 0.7387
Epoch 5/10
485/485 [==============================] - 4s 8ms/step - loss: 2.8318 - reg_acc: 0.8294 - val_loss: 4.4729 - val_reg_acc: 0.7488
Epoch 6/10
485/485 [==============================] - 4s 8ms/step - loss: 2.6255 - reg_acc: 0.8392 - val_loss: 5.4090 - val_reg_acc: 0.7415
Epoch 7/10
485/485 [==============================] - 4s 9ms/step - loss: 2.4856 - reg_acc: 0.8479 - val_loss: 4.4461 - val_reg_acc: 0.7440
Epoch 8/10
485/485 [

In [263]:
print(classification_report(y_test_array, np.round(np.clip(model.predict(X_test_array), 0, 4))))

              precision    recall  f1-score   support

         0.0       0.54      0.60      0.56       517
         1.0       0.10      0.13      0.11       148
         2.0       0.21      0.19      0.20       262
         3.0       0.22      0.25      0.23       461
         4.0       0.92      0.89      0.90      3777

    accuracy                           0.75      5165
   macro avg       0.40      0.41      0.40      5165
weighted avg       0.76      0.75      0.75      5165



In [265]:
dict_metrics['regression'] = np.mean(np.round(np.clip(model.predict(X_test_array), 0, 4))==y_test_array[:, np.newaxis])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  """Entry point for launching an IPython kernel.


Резлуьтаты конечно не улучшились, но зато стало понятно, что так подойти к задаче тоже можно!

#### Вывод:

In [269]:
list(sorted(dict_metrics.items(), key=lambda x: -x[1]))

[('RNN_lstm_seq', 0.7539206147193909),
 ('regression', 0.745595353339787),
 ('RNN_lstm_seq_drop', 0.7455953359603882),
 ('RNN_lstm_there_back', 0.7446272969245911),
 ('baseline', 0.7444337010383606),
 ('CNN_ker3', 0.7430784106254578),
 ('RNN_gru', 0.7395934462547302),
 ('RNN_CNN', 0.7368828654289246),
 ('RNN_rnn_seq', 0.7364956140518188),
 ('CNN_RNN', 0.7357211709022522),
 ('CNN_ker2_ker3', 0.7353339791297913),
 ('RNN_lstm', 0.7312681674957275),
 ('RNN_lstm_seq_2lay', 0.727202296257019),
 ('RNN_rnn', 0.7260406613349915),
 ('log_reg', 0.7130687318489836)]

И того: условно лучшие варианты - это lstm с усреднением всех выходов и решение задачи регресии с loss MAE(MSE показал себя гораздо хуже), условно худшие - это rnn и логистическая регресия.

Стоит отметить, что CNN гораздо быстрее RNN.

Но основная проблема, это переобучение, и если почти любую из этих моделей обучать меньшее количество эпох, то метрика будет гораздо выше, например:

In [274]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    Conv1D(embedding_dim*2, 3),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [275]:
print(model.summary())
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              loss_weights=disbalance,
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_49 (Embedding)    (None, None, 300)         2719500   
                                                                 
 conv1d_38 (Conv1D)          (None, None, 600)         540600    
                                                                 
 global_max_pooling1d_49 (Gl  (None, 600)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_147 (Dense)           (None, 512)               307712    
                                                                 
 dense_148 (Dense)           (None, 256)               131328    
                                                                 
 dense_149 (Dense)           (None, 5)                 1285      
                                                     

In [276]:
with tf.device("GPU:0"):
    hh = model.fit(X_train_array, y_train_array, validation_data=(X_test_array, y_test_array), epochs=2)

Epoch 1/2
485/485 [==============================] - 4s 8ms/step - loss: 6.9095 - accuracy: 0.7627 - val_loss: 6.6745 - val_accuracy: 0.7741
Epoch 2/2
485/485 [==============================] - 3s 7ms/step - loss: 5.5080 - accuracy: 0.8055 - val_loss: 6.4731 - val_accuracy: 0.7741


*** В этот момент я заметил, что до этого везде неправильно использовал функцию classification_report, так как сначала должно идти y_true, а потом y_pred. А до этого момента у меня везде было наоборот. На ключевую метрику accuracy это не влияет, но на остальные влияет, конечно обидно, что так вышло. Я конечно исправил везде, но выводы новые уже не смогу получить, так как перезапускать ноутбук очень долго из-за моделей с RNN слоями. ***

In [286]:
print(classification_report(y_test_array, np.argmax(model.predict(X_test_array), axis=1)))

              precision    recall  f1-score   support

           0       0.53      0.74      0.62       577
           1       0.09      0.01      0.01       190
           2       0.24      0.15      0.19       234
           3       0.32      0.16      0.22       511
           4       0.87      0.95      0.91      3653

    accuracy                           0.77      5165
   macro avg       0.41      0.40      0.39      5165
weighted avg       0.72      0.77      0.74      5165



In [287]:
hh.history['val_accuracy'][-1]

0.7740561366081238

Так что в этой задаче больше стоит уделить внимания данным, а не подбору архитектуры модели!

## Попробуем новую предобработку текста

Посмотрим на частые тэги из nltk

In [522]:
tags = []
for text in X_train['Content']:
  text = str(text)
  text = text.lower()
  tags.extend([i[1] for i in nltk.pos_tag(nltk.word_tokenize(text))])
tags_count = Counter(tags)
tags_count.most_common(30)

[('NNP', 81287),
 ('NN', 23588),
 ('.', 17694),
 ('JJ', 16145),
 (',', 8663),
 ('VB', 3338),
 ('CD', 1206),
 (')', 1204),
 (':', 1095),
 ('CC', 520),
 ('(', 436),
 ('VBD', 417),
 ("''", 295),
 ('``', 278),
 ('VBP', 257),
 ('FW', 213),
 ('VBZ', 161),
 ('$', 106),
 ('RB', 97),
 ('SYM', 83),
 ('NNS', 82),
 ('IN', 38),
 ('DT', 31),
 ('UH', 22),
 ('PRP', 17),
 ('POS', 14),
 ('JJS', 8),
 ('#', 7),
 ('VBG', 7),
 ('MD', 7)]

Как выяснилось, nltk не всегда точно определяет тэг, к тому же так получается, что метрика лучше, если ничего не убирать. В теории можно было бы перебрать все тэги, по очереди заменяя слова тега на название тега и если метрика увеличивается, то так и оставлять. Но это очень долго. Но всё же попробуем заменять некоторые тэги:

Давайте заменим слова с низкочастотными тэгами на их тэги:

In [638]:
top_n_tag = list(dict(tags_count.most_common(30)).keys())[:17]

In [639]:
def preprocess_text(txt):
    txt = str(txt)
    txt = txt.lower()
    txt = [word[1] if word[1] not in top_n_tag else word[0] for word in nltk.pos_tag(nltk.word_tokenize(txt))]
    txt = "".join(c for c in " ".join(txt) if c not in {'.', ','}) # уберём знаки припепнания, так как они не несут смысла
    return txt

X_train['Content_preprocess_new'] = X_train['Content'].apply(preprocess_text)
X_test['Content_preprocess_new'] = X_test['Content'].apply(preprocess_text)
X_train.head()

,Content,Date,Content_preprocess,Content_sequence,Content_preprocess_new,Content_sequence_new
6926,Отличное приложение)))),2017-08-04,отличный приложение,"[8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",отличное приложение ) ) ) ),"[25, 1, 13, 13, 13, 13, 0, 0, 0, 0, 0, 0, 0, 0..."
6735,Олекум,2017-08-04,олекума,"[3391, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",олекум,"[5308, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
149,"Хорошее приложение, но «антивирус» очень загру...",2017-08-14,хороший приложение « антивирус » загружать сис...,"[7, 1, 1016, 16, 1017, 595, 199, 0, 0, 0, 0, 0...",хорошее приложение но « антивирус » очень заг...,"[39, 1, 19, 1627, 59, 1473, 4, 1035, 507, 0, 0..."
3069,Потянет,2017-08-09,потянуть,"[2426, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",потянет,"[3656, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17455,Ужасное приложение положил деньги и они испари...,2017-07-11,ужасный приложение положить деньга испариться ...,"[211, 1, 534, 36, 3392, 773, 356, 472, 1633, 8...",ужасное приложение положил деньги и они испари...,"[414, 1, 2041, 96, 3, 294, 5309, 38, 5310, 480..."


In [640]:
corpus = (" ".join(X_train['Content_preprocess_new'])).split()
corpus_count = Counter(corpus)
corpus_count.most_common(10)

[('!', 5937),
 ('приложение', 4046),
 ('не', 3838),
 ('и', 3689),
 ('очень', 2874),
 ('?', 2825),
 ('удобно', 2622),
 ('все', 2590),
 ('в', 2277),
 ('на', 1530)]

In [641]:
max_words_true = len(corpus_count)
max_words_true

13593

In [642]:
MAX_WORDS = max_words_true
MAX_WORDS

13593

In [643]:
vocabulary = {j:i for i, (j, _) in enumerate(corpus_count.most_common()[:MAX_WORDS], 1)}
list(vocabulary.items())[:10]

[('!', 1),
 ('приложение', 2),
 ('не', 3),
 ('и', 4),
 ('очень', 5),
 ('?', 6),
 ('удобно', 7),
 ('все', 8),
 ('в', 9),
 ('на', 10)]

In [644]:
max_len_true = max([len(txt) for txt in X_train['Content_preprocess_new']])
max_len_true

1159

In [645]:
np.mean([len(txt) for txt in X_train['Content_preprocess_new']])

56.62475797082742

Берём максимальную длину

In [646]:
MAX_LEN = max_len_true
MAX_LEN

1159

In [647]:
X_train['Content_sequence_new'] = X_train['Content_preprocess_new'].apply(lambda text: [vocabulary.get(word, 0) for word in text.split()][:MAX_LEN]+[0]*(MAX_LEN-len(text.split())))
X_test['Content_sequence_new'] = X_test['Content_preprocess_new'].apply(lambda text: [vocabulary.get(word, 0) for word in text.split()][:MAX_LEN]+[0]*(MAX_LEN-len(text.split())))
X_train.head()

,Content,Date,Content_preprocess,Content_sequence,Content_preprocess_new,Content_sequence_new
6926,Отличное приложение)))),2017-08-04,отличный приложение,"[8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",отличное приложение ) ) ) ),"[26, 2, 14, 14, 14, 14, 0, 0, 0, 0, 0, 0, 0, 0..."
6735,Олекум,2017-08-04,олекума,"[3391, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",олекум,"[5365, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
149,"Хорошее приложение, но «антивирус» очень загру...",2017-08-14,хороший приложение « антивирус » загружать сис...,"[7, 1, 1016, 16, 1017, 595, 199, 0, 0, 0, 0, 0...",хорошее приложение но « антивирус » очень заг...,"[39, 2, 20, 1498, 59, 1499, 5, 993, 521, 0, 0,..."
3069,Потянет,2017-08-09,потянуть,"[2426, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",потянет,"[3705, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17455,Ужасное приложение положил деньги и они испари...,2017-07-11,ужасный приложение положить деньга испариться ...,"[211, 1, 534, 36, 3392, 773, 356, 472, 1633, 8...",ужасное приложение положил деньги и они испари...,"[426, 2, 2073, 96, 4, 299, 5366, 38, 5367, 498..."


In [648]:
X_train_array = np.array([[j for j in i] for i in X_train['Content_sequence_new']])
X_test_array = np.array([[j for j in i] for i in X_test['Content_sequence_new']])
X_train_array[0][:10]

array([26,  2, 14, 14, 14, 14,  0,  0,  0,  0])

### Модель

Используем CNN, так как она самая быстрая и с ней лучше, чем без неё.

In [665]:
embedding_dim=300
hidden_dim=512

model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim, mask_zero=True),
    Conv1D(embedding_dim*2, 3),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [666]:
print(model.summary())
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              loss_weights=disbalance,
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_76"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_76 (Embedding)    (None, None, 300)         4078200   
                                                                 
 conv1d_63 (Conv1D)          (None, None, 600)         540600    
                                                                 
 global_max_pooling1d_75 (Gl  (None, 600)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_228 (Dense)           (None, 512)               307712    
                                                                 
 dense_229 (Dense)           (None, 256)               131328    
                                                                 
 dense_230 (Dense)           (None, 5)                 1285      
                                                     

In [662]:
with tf.device("GPU:0"):
    hh = model.fit(X_train_array, y_train_array, validation_data=(X_test_array, y_test_array), epochs=2)

Epoch 1/2
485/485 [==============================] - 28s 57ms/step - loss: 6.7255 - accuracy: 0.7652 - val_loss: 6.1693 - val_accuracy: 0.7863
Epoch 2/2
485/485 [==============================] - 27s 56ms/step - loss: 5.1452 - accuracy: 0.8187 - val_loss: 6.4666 - val_accuracy: 0.7791


In [663]:
print(classification_report(y_test_array, np.argmax(model.predict(X_test_array), axis=1)))

              precision    recall  f1-score   support

           0       0.57      0.71      0.63       577
           1       0.20      0.02      0.03       190
           2       0.23      0.22      0.22       234
           3       0.35      0.18      0.24       511
           4       0.88      0.95      0.91      3653

    accuracy                           0.78      5165
   macro avg       0.45      0.41      0.41      5165
weighted avg       0.74      0.78      0.75      5165



In [664]:
hh.history['val_accuracy'][-1]

0.7790900468826294

Метрику получилось немного поднять с помощью обьеденения малочастотных слов в тэги.

Так же можно попробывать n-грамный подход.